In [78]:
!pip install xgboost
!pip install model_selection

In [79]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.datasets import dump_svmlight_file
import numpy as np
from sklearn.metrics import precision_score
from google.cloud import storage, bigquery

In [80]:
# import some data to play with
iris = datasets.load_iris()
X = iris.data #[:, :2]  # we only take the first two features.
y = iris.target

In [83]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [85]:
print("Train data length:",len(X_train));
print("Test data length:",len(X_test));

Train data length: 120
Test data length: 30


In [86]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [98]:
parameters = {
    'eta': 0.3,  
#    'silent': True,  # option for logging
    'objective': 'multi:softprob',  # error evaluation for multiclass tasks
    'num_class': 3,  # number of classes to predic
    'max_depth': 3  # depth of the trees in the boosting process
    }  
num_round = 20  # the number of training iterations

In [99]:
bst = xgb.train(parameters, dtrain, num_round)

[16:58:53] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [89]:
preds = bst.predict(dtest)

In [90]:
preds[:5]

array([[0.00563805, 0.97755206, 0.01680987],
       [0.98254657, 0.01395848, 0.00349498],
       [0.0036375 , 0.00615226, 0.9902103 ],
       [0.00564738, 0.97917044, 0.0151822 ],
       [0.00540076, 0.93640935, 0.0581899 ]], dtype=float32)

In [91]:
preds = bst.predict(dtest)

In [92]:
best_preds = np.asarray([np.argmax(line) for line in preds])

In [93]:
best_preds

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [94]:
print(precision_score(y_test, best_preds, average='macro'))

1.0


In [95]:
model = 'model.bst'
bst.save_model(model)
bucket = storage.Client().bucket('netology_dataset_sample')
blob = bucket.blob('{}/{}'.format('models/iris04', model))
blob.upload_from_filename(model)